In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('train_01.csv', sep=',', nrows=200000);
print(df.shape)
df.head()

(200000, 4)


,Unnamed: 0,id,subcategory,price
0,0,3edeb34cf93f490ff760af85,914,199900
1,1,c98febd50dad3cc0ffc86085,2202,35000
2,2,ade01e13912a46a99134cc75,2204,30000
3,3,ab3e6941c11304c1519aef75,2209,80000
4,4,252452a91c944a22c276d995,2204,50000


In [3]:
test = pd.read_csv('test_01.csv', sep=',');
print(test.shape)
test.head()

(749525, 3)


,Unnamed: 0,id,subcategory
0,0,285ea2e9935ccdeb8378c6a5,603
1,1,adfb73820bbb831257df6e95,203
2,2,783025601c36202f633fc6a5,116
3,3,2f0cd2d2e15dc90afd847f95,1009
4,4,5c23a37902855a20172845a5,1104


In [4]:
y_label = 'price'

def prepare_features(df_input, target_feature):    

    df_output = df_input.copy()

    df_output = df_output[['subcategory',target_feature]] #id
    
       
    df_output = pd.get_dummies(df_output, columns=['subcategory'])   
       
    return df_output

In [5]:
df_clean = df.pipe(prepare_features, y_label)
print('df_clean', df_clean.shape)
df_clean.head()

df_clean (200000, 249)


,price,subcategory_101,subcategory_102,subcategory_104,subcategory_105,subcategory_106,subcategory_107,subcategory_108,subcategory_109,subcategory_111,...,subcategory_2208,subcategory_2209,subcategory_2210,subcategory_2211,subcategory_2212,subcategory_2213,subcategory_2214,subcategory_2215,subcategory_2216,subcategory_10001
0,199900,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,30000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,80000,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,50000,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
X = df_clean.drop([y_label], axis = 1)

y = df_clean[y_label]

print('X ', X.shape)
print('y ', y.shape)
print (type(X), type(y))

X  (200000, 248)
y  (200000,)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


In [7]:
import math

def rlmse_score(y_test, y_hat):
    y_test = [x if x>=0 else 0 for x in y_test]
    y_hat = [x if x>=0 else 0 for x in y_hat]
    score = 0
    for i in range(len(y_test)):
        score += (math.log(abs(y_test[i]+1)) - math.log(abs(y_hat[i]+1)))**2
    rlmse = (score/len(y_test))**0.5
    return rlmse

def rlmse_scorer(estimator, X, y):
    y_hat = estimator.predict(X)
    
    return rlmse_score(y, y_hat)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
model = RandomForestClassifier(n_estimators=80, n_jobs=-1, max_depth=5, criterion='entropy', min_samples_split=3, min_weight_fraction_leaf = 0.1)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.1, n_estimators=80, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [11]:
predict = model.predict(X_test)

In [13]:
rlmse_score(y_test, predict)

2.560191330035269

In [14]:
def prepare_test(df_input):    

    df_output = df_input.copy()
    df_output = df_output['subcategory'] #id
    
    df_output = pd.get_dummies(df_output, columns=['subcategory'])   
       
    return df_output

In [15]:
df_test = test.pipe(prepare_test)
print('df_test', df_test.shape)
df_test.head()

df_test (749525, 248)


,101,102,104,105,106,107,108,109,111,112,...,2209,2210,2211,2212,2213,2214,2215,2216,2301,10001
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df_test_split = np.array_split(df_test, 10)
print()

In [17]:
df_test_split[0].head()

,101,102,104,105,106,107,108,109,111,112,...,2209,2210,2211,2212,2213,2214,2215,2216,2301,10001
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
test_res_0 = model.predict(df_test_split[0])

In [19]:
test_res_1 = model.predict(df_test_split[1])

In [20]:
test_res_2 = model.predict(df_test_split[2])

In [21]:
test_res_3 = model.predict(df_test_split[3])

In [22]:
test_res_4 = model.predict(df_test_split[4])

In [23]:
test_res_5 = model.predict(df_test_split[5])

In [24]:
test_res_6 = model.predict(df_test_split[6])

In [25]:
test_res_7 = model.predict(df_test_split[7])

In [26]:
test_res_8 = model.predict(df_test_split[8])

In [27]:
test_res_9 = model.predict(df_test_split[9])

In [28]:
test_fin = np.concatenate((test_res_0, test_res_1, test_res_2, test_res_3, test_res_4, test_res_5, test_res_6, test_res_7, test_res_8, test_res_9), axis=0)

In [29]:
len(test_fin)

749525

In [30]:
data_out = test['id']
print(data_out.shape)
data_out.tail()

(749525,)


749520    776d94d55228402996c866a5
749521    429db1e7345402fcda4124a5
749522    a97cc12f186c1e2619fe56a5
749523    2f0262c2244588edae8fc2a5
749524    78e9c57b3320532f564b61a5
Name: id, dtype: object

In [31]:
df999 = pd.DataFrame(test_fin)
df999.head()

,0
0,50000
1,50000
2,50000
3,50000
4,50000


In [32]:
data_out = pd.concat([data_out,df999],axis=1)

In [33]:
data_out.columns = ['id', 'price']
data_out['price'] = data_out['price'].map(lambda x: x/100 )

In [34]:
print(data_out.shape)
data_out.head()

(749525, 2)


,id,price
0,285ea2e9935ccdeb8378c6a5,500.0
1,adfb73820bbb831257df6e95,500.0
2,783025601c36202f633fc6a5,500.0
3,2f0cd2d2e15dc90afd847f95,500.0
4,5c23a37902855a20172845a5,500.0


In [35]:
data_out.to_csv('hack_06.csv', index=False)